In [12]:
from ortools.linear_solver import pywraplp

In [21]:
# Define available resources
resources = ['A', 'B', 'C']

# Define the intents.
# For have, the first value is the amount we're willing to trade and the second is how much the trader values that resource if its possible to avoid trading all of them.
intents = [
    {'weight': 1.5, 'have': {'A': (3, 2.0), 'B': (4, 1.0)}, 'want': {'C': 2}},
    {'weight': 2.0, 'have': {'C': (2, 1.0)}, 'want': {'A': 1, 'B': 3}},
]

In [22]:
# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Create variables and constraints for each intent
objective = solver.Objective()
x_vars = [] # List to store the binary variables for each intent
for intent in intents:
    x = solver.BoolVar('') # Binary variable for the intent
    x_vars.append(x)
    objective.SetCoefficient(x, intent['weight'])

    maximal_trade_expr = sum(value * amount for resource, (amount, value) in intent['have'].items())
    actual_trade_expr = 0
    
    # Outflow variables and constraints for each resource
    for resource, amount in intent['want'].items():
        inflow = solver.IntVar(0, amount, '')
        solver.Add(inflow == amount * x)
    
    # Outflow variables and constraints for each resource
    outflow_vars_for_intent = {} # Dictionary to store outflow variables for this intent
    for resource, (amount, value) in intent['have'].items():
        outflow = solver.IntVar(0, amount, '')
        solver.Add(outflow >= 0)
        solver.Add(outflow <= amount * x)
        outflow_vars_for_intent[resource] = outflow
        actual_trade_expr += outflow * value
    
    # Minimize traded items weighted by intent preferences
    ratio = solver.NumVar(0, 1, 'ratio')
    solver.Add(ratio == actual_trade_expr / maximal_trade_expr)
    objective.SetCoefficient(ratio, -0.5)
    
    intent['outflow_vars'] = outflow_vars_for_intent # Add to the intent dictionary

# Constraints for conservation of flow
for resource in resources:
    outflow_expr = sum(intent['want'].get(resource, 0) * x for intent, x in zip(intents, x_vars))
    inflow_expr = sum(intent['outflow_vars'][resource] for intent in intents if resource in intent['outflow_vars'])
    solver.Add(outflow_expr == inflow_expr)

# Set the objective
objective.SetMaximization()

In [23]:
# Solve the problem
status = solver.Solve()

In [24]:
# Print the results
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for i, intent in enumerate(intents):
        satisfied = x_vars[i].solution_value()
        print(f"Intent {i}: {'Satisfied' if satisfied else 'Not satisfied'}")
        if satisfied:
            print("Outflows:")
            for resource, outflow in intent['outflow_vars'].items():
                print(f"  {resource}: {outflow.solution_value()}")
else:
    print('The problem does not have an optimal solution.')

Objective value = 2.75
Intent 0: Satisfied
Outflows:
  A: 1.0
  B: 3.0
Intent 1: Satisfied
Outflows:
  C: 2.0
